In [9]:
import config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import simplejson
from math import isnan

from pandas_profiling import ProfileReport
from ipywidgets import interact, Dropdown

In [10]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]
metaCollection = stagingDb['Kolominformatie']

In [13]:
#df = pd.DataFrame(list(analyseCol.find({'soort': 'Stelling'})))
df = pd.DataFrame(list(analyseCol.find({'soort': 'Put'})))
df.drop(['ID', 'index', 'level_0'], inplace=True, errors='ignore')
df.reset_index(inplace=True)
df['ID'] = df['index']
df

,index,_id,brondata,projectcd,putnr,soort,key,key_project,ID
0,0,620579e46026534a675d270d,"{'_id': 620579e46026534a675d270d, 'PUT': 2, 'P...",DC154,2,Put,PDC154P2,PDC154,0
1,1,620579e46026534a675d270e,"{'_id': 620579e46026534a675d270e, 'PUT': 1, 'P...",DC154,1,Put,PDC154P1,PDC154,1
2,2,620579f50b22202220526c58,"{'_id': 620579f50b22202220526c58, 'PUT': 1, 'P...",DC179,1,Put,PDC179P1,PDC179,2
3,3,620579f50b22202220526c59,"{'_id': 620579f50b22202220526c59, 'PUT': 2, 'P...",DC179,2,Put,PDC179P2,PDC179,3
4,4,620579f50b22202220526c5a,"{'_id': 620579f50b22202220526c5a, 'PUT': 3, 'P...",DC179,3,Put,PDC179P3,PDC179,4
5,5,620579f50b22202220526c5b,"{'_id': 620579f50b22202220526c5b, 'PUT': 4, 'P...",DC179,4,Put,PDC179P4,PDC179,5
6,6,620579f50b22202220526c5c,"{'_id': 620579f50b22202220526c5c, 'PUT': 5, 'P...",DC179,5,Put,PDC179P5,PDC179,6
7,7,620579f50b22202220526c5d,"{'_id': 620579f50b22202220526c5d, 'PUT': 6, 'P...",DC179,6,Put,PDC179P6,PDC179,7
8,8,620579f50b22202220526c5e,"{'_id': 620579f50b22202220526c5e, 'PUT': 7, 'P...",DC179,7,Put,PDC179P7,PDC179,8
9,9,620579f50b22202220526c5f,"{'_id': 620579f50b22202220526c5f, 'PUT': 8, 'P...",DC179,8,Put,PDC179P8,PDC179,9


In [87]:
mapper = {"$arrayToObject" : {"$filter": {"input" : {"$objectToArray" : "$brondata"}, 
                                          "as" : "item", 
                                          "cond" : {"$and" : [{"$ne" : ["$$item.v",np.NaN]},{"$ne" : ["$$item.v",None]}, {"$ne" : ["$$item.v",""]}]}}}}

df_count = pd.DataFrame(list(analyseCol.aggregate([
    {"$match": {"brondata": {"$exists": {"$Bool": 1}}, "soort": {"$exists": {"$Bool": 1}}}},
    {"$replaceRoot": { "newRoot": { "$mergeObjects": [ { "soort": {"$ifNull": ["$artefactsoort", "$soort"]}}, mapper ] } } }])))
df_count = df_count.groupby(['soort']).agg(['count'])
df_count.columns = list(df_count.columns.levels[0])
df_count = df_count.reset_index(level=0)
df_count = df_count.melt(id_vars=["soort", "_id"])
df_count = df_count[df_count.value != 0]
df_count.rename(columns={'value': 'teller'}, inplace=True)
df_count.sort_values(by=['soort', 'variable'], inplace=True)
df_count['percentage'] = pd.to_numeric(100 * df_count['teller'] / df_count['_id'], downcast='integer').round(0)
df_count.drop(columns=['_id'], inplace=True)
df_count.head(50)

,soort,variable,teller,percentage
1752,Aardewerk,10a,5196,75.0
2664,Aardewerk,10b,420,6.0
3600,Aardewerk,10c,190,3.0
3624,Aardewerk,10d,216,3.0
1776,Aardewerk,11,5918,85.0
1800,Aardewerk,12,6048,87.0
1824,Aardewerk,13,6048,87.0
2688,Aardewerk,1b,5082,73.0
2400,Aardewerk,1c,324,5.0
2424,Aardewerk,2,6046,87.0


In [68]:
df[df.value != 0]


,soort,variable,value
0,Aardewerk,_id,6953
1,Artefact,_id,175
2,Bot,_id,4
3,Doos,_id,2773
4,Glas,_id,461
...,...,...,...
9359,Vondst,VULLING,174
9383,Vondst,VAKNR,174
9407,Vondst,VERZMWIJZE,174
9431,Vondst,VAKLETTER,174


['_id',
 'brondata',
 'artefactnr',
 'opmerkingen',
 'typevoorwerp',
 'functievoorwerp',
 'literatuur',
 'tekeningnr',
 'soort',
 'artefactsoort',
 'key',
 'key_project',
 'fileName',
 'imageUUID',
 'imageMiddleUUID',
 'imageThumbUUID',
 'fileType',
 'directory',
 'mime_type',
 'fototype',
 'fotonr',
 'fotosubnr',
 'materiaal',
 'key_project_type',
 'putnr',
 'vondstnr',
 'key_artefact',
 'key_vondst',
 'beschrijving',
 'glassoort',
 'projectnaam',
 'doosnr',
 'uitgeleend',
 'inhoud',
 'datering',
 'publicatiecode',
 'dateringvanaf',
 'dateringtot',
 'typecd',
 'restauratie',
 'exposabel',
 'key_doos',
 'key_plaatsing',
 'key_put',
 'origine',
 'dianr',
 'datering_origineel',
 'vlaknr',
 'stelling',
 'table',
 'herkomst',
 'key_stelling',
 'spoornr',
 'vorm',
 'diepte',
 'key_vlak',
 'error',
 'volgletter',
 'aard',
 'vaknr',
 'omstandigheden',
 'datum',
 'conserveren',
 'graf',
 'houtsoortcd',
 'maten',
 'bewerkingssporen',
 'gebruikssporen',
 'decoratie',
 'key_standplaats',
 'toponi